# Credit Score Analysis

## Business Problem

### Problem Statement

Home Credit Indonesia want to know which customer are less risky to pay the credit amount. As a Data Scientist, the company want us to predict which customer is in good or bad category based on their data and characteristic.

### Analytical Approach

Classification Machine learning will be used to determine which customer are accepted to pay with credit by using ROC/AUC performance.

## Data Understanding

### Library

In [209]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Information Value
import scipy.stats as stats
from pandas.core.dtypes.common import is_numeric_dtype

from sklearn.model_selection import GridSearchCV,StratifiedKFold,train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.metrics import roc_curve, roc_auc_score

# Ignore Warning
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### Dataset

The datasets (train and test) are already in all numeric from the previous process.

#### Data Train

In [210]:
df = pd.read_csv('CS_train.csv')
df

,Unnamed: 0,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,Bureau_CREDIT_ACTIVE,Bureau_CREDIT_CURRENCY,Bureau_DAYS_CREDIT,Bureau_CREDIT_DAY_OVERDUE,Bureau_DAYS_CREDIT_ENDDATE,Bureau_DAYS_ENDDATE_FACT,Bureau_AMT_CREDIT_MAX_OVERDUE,Bureau_CNT_CREDIT_PROLONG,Bureau_AMT_CREDIT_SUM,Bureau_AMT_CREDIT_SUM_DEBT,Bureau_AMT_CREDIT_SUM_LIMIT,Bureau_AMT_CREDIT_SUM_OVERDUE,Bureau_CREDIT_TYPE,Bureau_DAYS_CREDIT_UPDATE,Bureau_AMT_ANNUITY,Bureau_Balance_MONTHS_BALANCE,Bureau_Balance_STATUS_C,Bureau_Balance_STATUS_0,Bureau_Balance_STATUS_X,Bureau_Balance_STATUS_1,Bureau_Balance_STATUS_2,Bureau_Balance_STATUS_3,Bureau_Balance_STATUS_5,Bureau_Balance_STATUS_4,Prev_AMT_ANNUITY,Prev_AMT_APPLICATION,Prev_AMT_CREDIT,Prev_AMT_DOWN_PAYMENT,Prev_AMT_GOODS_PRICE,Prev_HOUR_APPR_PROCESS_START,Prev_FLAG_LAST_APPL_PER_CONTRACT,Prev_NFLAG_LAST_APPL_IN_DAY,Prev_RATE_DOWN_PAYMENT,Prev_RATE_INTEREST_PRIMARY,Prev_RATE_INTEREST_PRIVILEGED,Prev_DAYS_DECISION,Prev_SELLERPLACE_AREA,Prev_CNT_PAYMENT,Prev_DAYS_FIRST_DRAWING,Prev_DAYS_FIRST_DUE,Prev_DAYS_LAST_DUE_1ST_VERSION,Prev_DAYS_LAST_DUE,Prev_DAYS_TERMINATION,Prev_NFLAG_INSURED_ON_APPROVAL,Prev_NAME_CONTRACT_TYPE_Consumer loans,Prev_NAME_CONTRACT_TYPE_Cash loans,Prev_NAME_CONTRACT_TYPE_Revolving loans,Prev_NAME_CONTRACT_TYPE_XNA,Prev_WEEKDAY_APPR_PROCESS_START_SATURDAY,Prev_WEEKDAY_APPR_PROCESS_START_THURSDAY,Prev_WEEKDAY_APPR_PROCESS_START_TUESDAY,Prev_WEEKDAY_APPR_PROCESS_START_MONDAY,Prev_WEEKDAY_APPR_PROCESS_START_FRIDAY,Prev_WEEKDAY_APPR_PROCESS_START_SUNDAY,Prev_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,Prev_NAME_CASH_LOAN_PURPOSE_XAP,Prev_NAME_CASH_LOAN_PURPOSE_XNA,Prev_NAME_CASH_LOAN_PURPOSE_Repairs,Prev_NAME_CASH_LOAN_PURPOSE_Everyday expenses,Prev_NAME_CASH_LOAN_PURPOSE_Car repairs,Prev_NAME_CASH_LOAN_PURPOSE_Building a house or an annex,Prev_NAME_CASH_LOAN_PURPOSE_Other,Prev_NAME_CASH_LOAN_PURPO

In [211]:
df.shape

(307511, 343)

#### Data Test

In [212]:
df_test = pd.read_csv('CS_test.csv')
df_test

,Unnamed: 0,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,Bureau_CREDIT_ACTIVE,Bureau_CREDIT_CURRENCY,Bureau_DAYS_CREDIT,Bureau_CREDIT_DAY_OVERDUE,Bureau_DAYS_CREDIT_ENDDATE,Bureau_DAYS_ENDDATE_FACT,Bureau_AMT_CREDIT_MAX_OVERDUE,Bureau_CNT_CREDIT_PROLONG,Bureau_AMT_CREDIT_SUM,Bureau_AMT_CREDIT_SUM_DEBT,Bureau_AMT_CREDIT_SUM_LIMIT,Bureau_AMT_CREDIT_SUM_OVERDUE,Bureau_CREDIT_TYPE,Bureau_DAYS_CREDIT_UPDATE,Bureau_AMT_ANNUITY,Bureau_Balance_MONTHS_BALANCE,Bureau_Balance_STATUS_C,Bureau_Balance_STATUS_0,Bureau_Balance_STATUS_X,Bureau_Balance_STATUS_1,Bureau_Balance_STATUS_2,Bureau_Balance_STATUS_3,Bureau_Balance_STATUS_5,Bureau_Balance_STATUS_4,Prev_AMT_ANNUITY,Prev_AMT_APPLICATION,Prev_AMT_CREDIT,Prev_AMT_DOWN_PAYMENT,Prev_AMT_GOODS_PRICE,Prev_HOUR_APPR_PROCESS_START,Prev_FLAG_LAST_APPL_PER_CONTRACT,Prev_NFLAG_LAST_APPL_IN_DAY,Prev_RATE_DOWN_PAYMENT,Prev_RATE_INTEREST_PRIMARY,Prev_RATE_INTEREST_PRIVILEGED,Prev_DAYS_DECISION,Prev_SELLERPLACE_AREA,Prev_CNT_PAYMENT,Prev_DAYS_FIRST_DRAWING,Prev_DAYS_FIRST_DUE,Prev_DAYS_LAST_DUE_1ST_VERSION,Prev_DAYS_LAST_DUE,Prev_DAYS_TERMINATION,Prev_NFLAG_INSURED_ON_APPROVAL,Prev_NAME_CONTRACT_TYPE_Consumer loans,Prev_NAME_CONTRACT_TYPE_Cash loans,Prev_NAME_CONTRACT_TYPE_Revolving loans,Prev_NAME_CONTRACT_TYPE_XNA,Prev_WEEKDAY_APPR_PROCESS_START_SATURDAY,Prev_WEEKDAY_APPR_PROCESS_START_THURSDAY,Prev_WEEKDAY_APPR_PROCESS_START_TUESDAY,Prev_WEEKDAY_APPR_PROCESS_START_MONDAY,Prev_WEEKDAY_APPR_PROCESS_START_FRIDAY,Prev_WEEKDAY_APPR_PROCESS_START_SUNDAY,Prev_WEEKDAY_APPR_PROCESS_START_WEDNESDAY,Prev_NAME_CASH_LOAN_PURPOSE_XAP,Prev_NAME_CASH_LOAN_PURPOSE_XNA,Prev_NAME_CASH_LOAN_PURPOSE_Repairs,Prev_NAME_CASH_LOAN_PURPOSE_Everyday expenses,Prev_NAME_CASH_LOAN_PURPOSE_Car repairs,Prev_NAME_CASH_LOAN_PURPOSE_Building a house or an annex,Prev_NAME_CASH_LOAN_PURPOSE_Other,Prev_NAME_CASH_LOAN_PURPOSE_Jour

In [213]:
df_test.shape

(48744, 342)

## Data Cleaning

### Irrelevant Column

Column `Unnamed: 0` and `SK_ID_CURR` are not going to used on data modeling.

In [214]:
df.drop(['Unnamed: 0', 'SK_ID_CURR'], axis=1, inplace=True)
df_test.drop(['Unnamed: 0', 'SK_ID_CURR'], axis=1, inplace=True)

### Missing Value

In [215]:
missing = pd.DataFrame(df.isna().sum() * 100 / len(df))

In [216]:
missing[missing[0]>30]

,0
OWN_CAR_AGE,65.990810
OCCUPATION_TYPE,31.345545
EXT_SOURCE_1,56.381073
APARTMENTS_AVG,50.749729
BASEMENTAREA_AVG,58.515956
...,...
Prev_CCB_CNT_DRAWINGS_POS_CURRENT,73.977516
Prev_CCB_CNT_INSTALMENT_MATURE_CUM,61.247890
Prev_CCB_NAME_CONTRACT_STATUS,61.247890
Prev_CCB_SK_DPD,61.247890


Drop columns with more than 30% missing value

In [217]:
drop_col = missing[missing[0]>30]
df.drop(columns=drop_col.index, inplace=True)
df.shape

(307511, 257)

Input the other NaN with median.

In [218]:
for feature in missing[(missing[0] > 0) & (missing[0] <= 30)].index:
    df[feature] = df[feature].fillna(df[feature].median())

In [219]:
df.isna().sum().sum()

0

## Data Preparation

### Information Value

#### Code

In [220]:
#categoric fitur dan continuous fiture

class CategoricalFeature():
    def __init__(self, df, feature):
        self.df = df
        self.feature = feature

    @property
    def df_lite(self):
        df_lite = self.df
        df_lite['bin'] = df_lite[self.feature].fillna('MISSING')
        return df_lite[['bin', 'TARGET']]


class ContinuousFeature():
    def __init__(self, df, feature):
        self.df = df
        self.feature = feature
        self.bin_min_size = int(len(self.df) * 0.05)

    def __generate_bins(self, bins_num):
        df = self.df[[self.feature, 'TARGET']]
        df['bin'] = pd.qcut(df[self.feature], bins_num, duplicates='drop') \
                    .apply(lambda x: x.left) \
                    .astype(float)
        return df

    def __generate_correct_bins(self, bins_max=20):
        for bins_num in range(bins_max, 1, -1):
            df = self.__generate_bins(bins_num)
            df_grouped = pd.DataFrame(df.groupby('bin') \
                                      .agg({self.feature: 'count',
                                            'TARGET': 'sum'})) \
                                      .reset_index()
            r, p = stats.stats.spearmanr(df_grouped['bin'], df_grouped['TARGET'])

            if (
                    abs(r)==1 and                                                        # periksa apakah WOE untuk tiap bin adalah monotomic
                    df_grouped[self.feature].min() > self.bin_min_size                   # periksa apakah size setiap bin besar dari 5%
                    and not (df_grouped[self.feature] == df_grouped['TARGET']).any()      # periksa apakah label Yes dan No jumlahnya adalah 0
            ):
                break

        return df

    @property
    def df_lite(self):
        df_lite = self.__generate_correct_bins()
        df_lite['bin'].fillna('MISSING', inplace=True)
        return df_lite[['bin', 'TARGET']]

In [221]:
# Information Value
pd.set_option('mode.chained_assignment', None)

class AttributeRelevance():
    def seq_palette(self, n_colors):
        return sns.cubehelix_palette(n_colors, start=.5, rot=-.75, reverse=True)

    def bulk_iv(self, feats, iv, woe_extremes=False):
        iv_dict = {}
        for f in feats:
            iv_df, iv_value = iv.calculate_iv(f)
            if woe_extremes:
                iv_dict[f.feature] = [iv_value, iv_df['woe'].min(), iv_df['woe'].max()]
                cols = ['iv', 'woe_min', 'woe_max']
            else:
                iv_dict[f.feature] = iv_value
                cols = ['iv']
        df = pd.DataFrame.from_dict(iv_dict, orient='index', columns=cols)
        return df

    def bulk_stats(self, feats, s):
        stats_dict = {}
        for f in feats:
            p_value, effect_size = s.calculate_chi(f)
            stats_dict[f.feature] = [p_value, effect_size]
        df = pd.DataFrame.from_dict(stats_dict, orient='index', columns=['p-value', 'effect_size'])
        return df

    def analyze(self, feats, iv, s=None, interpretation=False):
        df_iv = self.bulk_iv(feats, iv).sort_values(by='iv', ascending=False)
        if s is not None:
            df_stats = self.bulk_stats(feats, s)
            df_iv = df_iv.merge(df_stats, left_index=True, right_index=True)
        if interpretation:
            df_iv['iv_interpretation'] = df_iv['iv'].apply(iv.interpretation)
            if s is not None:
                df_iv['es_interpretation'] = df_iv['effect_size'].apply(s.interpretation)
        return df_iv

    def draw_iv(self, feats, iv):
        df = self.analyze(feats, iv)
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(x=df.index, y='iv', data=df, palette=self.seq_palette(len(feats)))
        ax.set_title('IV values')
        plt.xticks(rotation=90)
        plt.show()

    def draw_woe_extremes(self, feats, iv):
        df = self.bulk_iv(feats, iv, woe_extremes=True).sort_values(by='iv', ascending=False)
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.barplot(x=df.index, y='woe_min', data=df, palette=self.seq_palette(len(feats)))
        sns.barplot(x=df.index, y='woe_max', data=df, palette=self.seq_palette(len(feats)))
        ax.axhline(y=0, color='black', linewidth=1)
        ax.set_title('Range of WOE values')
        ax.set_ylabel('WOE')
        plt.xticks(rotation=90)
        plt.show()

    def draw_woe_multiplot(self, feats, iv):
        n = len(feats)
        nrows = int(np.ceil(n/3))
        fig, ax = plt.subplots(nrows=nrows, ncols=3, figsize=(15, nrows*4))
        for i in range(n):
            iv_df, iv_value = iv.calculate_iv(feats[i])
            sns.barplot(x=feats[i].feature, y='woe', data=iv_df, color='#455872', ax=fig.axes[i])

        for ax in fig.axes:
            plt.sca(ax)
            plt.xticks(rotation=50)

        plt.tight_layout()
        plt.show()

class Analysis():
    def seq_palette(self, n_colors):
        return sns.cubehelix_palette(n_colors, start=.5, rot=-.75, reverse=True)

    def group_by_feature(self, feat):
        df = feat.df_lite \
                            .groupby('bin') \
                            .agg({'TARGET': ['count', 'sum']}) \
                            .reset_index()
        df.columns = [feat.feature, 'count', 'good']
        df['bad'] = df['count'] - df['good']
        return df

class StatsSignificance(Analysis):
    def calculate_chi(self, feat):
        df = self.group_by_feature(feat)
        df_chi = np.array(df[['good', 'bad']])
        n = df['count'].sum()

        chi = stats.chi2_contingency(df_chi)
        cramers_v = np.sqrt(chi[0] / n)          # assume that k=2 (good, bad)
        return chi[1], cramers_v

    @staticmethod
    def interpretation(cramers_v):
        if cramers_v < 0.1:
            return 'useless'
        elif cramers_v < 0.2:
            return 'weak'
        elif cramers_v < 0.4:
            return 'medium'
        elif cramers_v < 0.6:
            return 'strong'
        else:
            return 'very strong'

    def interpret_chi(self, feat):
        _, cramers_v = self.calculate_chi(feat)
        return self.interpretation(cramers_v)

    def print_chi(self, feat):
        p_value, cramers_v = self.calculate_chi(feat)
        print('P-value: %0.2f\nEffect size: %0.2f' % (p_value, cramers_v))
        print('%s is a %s predictor' % (feat.feature.capitalize(), self.interpretation(cramers_v)))


class IV(Analysis):
    @staticmethod
    def __perc_share(df, group_name):
        return df[group_name] / df[group_name].sum()

    def __calculate_perc_share(self, feat):
        df = self.group_by_feature(feat)
        df['perc_good'] = self.__perc_share(df, 'good')
        df['perc_bad'] = self.__perc_share(df, 'bad')
        df['perc_diff'] = df['perc_good'] - df['perc_bad']
        return df

    def __calculate_woe(self, feat):
        df = self.__calculate_perc_share(feat)
        df['woe'] = np.log(df['perc_good']/df['perc_bad'])
        df['woe'] = df['woe'].replace([np.inf, -np.inf], np.nan).fillna(0)
        return df

    def calculate_iv(self, feat):
        df = self.__calculate_woe(feat)
        df['iv'] = df['perc_diff'] * df['woe']
        return df, df['iv'].sum()

    def draw_woe(self, feat):
        iv_df, iv_value = self.calculate_iv(feat)
        fig, ax = plt.subplots(figsize=(10,6))
        sns.barplot(x=feat.feature, y='woe', data=iv_df, palette=self.seq_palette(len(iv_df.index)))
        ax.set_title('WOE visualization for: ' + feat.feature)
        plt.show()
        plt.show()

    @staticmethod
    def interpretation(iv):
        if iv < 0.02:
            return 'useless'
        elif iv < 0.1:
            return 'weak'
        elif iv < 0.3:
            return 'medium'
        elif iv < 0.5:
            return 'strong'
        else:
            return 'suspicious'

    def interpret_iv(self, feat):
        _, iv = self.calculate_iv(feat)
        return self.interpretation(iv)

    def print_iv(self, feat):
        _, iv = self.calculate_iv(feat)
        print('Information value: %0.2f' % iv)
        print('%s is a %s predictor' % (feat.feature.capitalize(), self.interpretation(iv)))

In [222]:
#selection feat
feats_dict = {}

for col in [c for c in df.columns if c != 'TARGET']:
  if is_numeric_dtype(df[col]):
    feats_dict[col] = ContinuousFeature(df,col)
  else :
    feats_dict[col] = CategoricalFeature(df,col)

feats = list(feats_dict.values())

#### Result

In [223]:
iv = IV()
s = StatsSignificance()

ar = AttributeRelevance()


df_analysis = ar.analyze(feats, iv, s, interpretation=True)
display(df_analysis)

,iv,p-value,effect_size,iv_interpretation,es_interpretation
EXT_SOURCE_2,0.318686,0.000000e+00,0.163345,strong,weak
EXT_SOURCE_3,0.229972,0.000000e+00,0.125682,medium,weak
DAYS_EMPLOYED,0.089364,0.000000e+00,0.080846,weak,useless
DAYS_BIRTH,0.084920,0.000000e+00,0.078413,weak,useless
NAME_INCOME_TYPE,0.057537,8.116436e-264,0.062941,weak,useless
...,...,...,...,...,...
Prev_NAME_CASH_LOAN_PURPOSE_Payments on other loans,0.000000,1.000000e+00,0.000000,useless,useless
Prev_NAME_CASH_LOAN_PURPOSE_Urgent needs,0.000000,1.000000e+00,0.000000,useless,useless
Prev_NAME_CASH_LOAN_PURPOSE_Buying a used car,0.000000,1.000000e+00,0.000000,useless,useless
Prev_NAME_CASH_LOAN_PURPOSE_Buying a new car,0.000000,1.000000e+00,0.000000,useless,useless


After looking on the Information Value, there are no suspicious columns that could lead to information leakage. But with 256 rows, the most useless columns will be removed.

### Data splitting

In [224]:
x = df.drop(columns=['TARGET'])
y = df['TARGET']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)

#### Scaling

In [225]:
x_train = MinMaxScaler().fit_transform(x_train)

## Modeling

In [226]:
lr = LogisticRegression(random_state=28)

In [227]:
models = [lr]
score=[]
rata=[]
std=[]

for i in models:
    skfold=StratifiedKFold(n_splits=5)
    estimator=Pipeline([
        ('model',i)])
    model_cv=cross_val_score(estimator,x_train,y_train,cv=skfold,scoring='roc_auc')
    score.append(model_cv)
    rata.append(model_cv.mean())
    std.append(model_cv.std())
    
pd.DataFrame({'model':['Logistic Regression'],'mean roc_auc':rata,'sdev':std}).set_index('model').sort_values(by='mean roc_auc',ascending=False)

,mean roc_auc,sdev
model,,
Logistic Regression,0.734877,0.004611


## Improvements

- Outliers handling
- Performing further feature selection
- Imbalance analysis
- Trying another Machine Learning Algorithm
- Performing Hyperparameter tuning